In [3]:
!ls ../../data/

memes_comments.20.19.log    memes_infections.20.20.log
memes_comments.20.20.log    memes_infections.20.21.log
memes_comments.20.21.log    memes_infections.20.22.log
memes_comments.20.22.log    memes_infections.20.23.log
memes_comments.20.23.log    memes_infections.21.0.log
memes_comments.21.0.log     memes_infections.21.10.log
memes_comments.21.10.log    memes_infections.21.11.log
memes_comments.21.11.log    memes_infections.21.1.log
memes_comments.21.1.log     memes_infections.21.2.log
memes_comments.21.2.log     memes_infections.21.3.log
memes_comments.21.3.log     memes_infections.21.4.log
memes_comments.21.4.log     memes_infections.21.5.log
memes_comments.21.5.log     memes_infections.21.6.log
memes_comments.21.6.log     memes_infections.21.7.log
memes_comments.21.7.log     memes_infections.21.8.log
memes_comments.21.8.log     memes_infections.21.9.log
memes_comments.21.9.log     README.md
memes_infections.20.19.log


In [63]:
# imports

%matplotlib inline

# import codecs
import pandas
import glob
from datetime import datetime
from collections import defaultdict
import numpy as np

# import networkx as nx
# import plotly.graph_objects as go
import plotly.express as px

In [64]:
glob.glob('../../data/memes_infections*.log')

['../../data/memes_infections.21.22.log',
 '../../data/memes_infections.21.23.log',
 '../../data/memes_infections.22.0.log',
 '../../data/memes_infections.22.1.log',
 '../../data/memes_infections.22.10.log',
 '../../data/memes_infections.22.11.log',
 '../../data/memes_infections.22.12.log',
 '../../data/memes_infections.22.13.log',
 '../../data/memes_infections.22.2.log',
 '../../data/memes_infections.22.3.log',
 '../../data/memes_infections.22.4.log',
 '../../data/memes_infections.22.5.log',
 '../../data/memes_infections.22.6.log',
 '../../data/memes_infections.22.7.log',
 '../../data/memes_infections.22.8.log',
 '../../data/memes_infections.22.9.log',
 '../../data/memes_infections.20.20.log',
 '../../data/memes_infections.20.21.log',
 '../../data/memes_infections.20.22.log',
 '../../data/memes_infections.20.23.log',
 '../../data/memes_infections.21.0.log',
 '../../data/memes_infections.21.1.log',
 '../../data/memes_infections.21.10.log',
 '../../data/memes_infections.21.11.log',
 '..

In [2]:
PATIENT_0 = 'u/woodendoors7'

In [65]:
# doc = codecs.open('../data/memes_infections.20.20.log','rU','UTF-16')
all_files = glob.glob('../../data/memes_infections*.log')
get_file = lambda fname: pandas.read_csv(
    fname,
    sep='\t',
    header=None,
    names=['post_dt',
            'infectee_name',
            'infectee_post_id',
            'infector_name',
            'infector_post_id',
            'comment_or_submission'],
    parse_dates=['post_dt']
)
df = pandas.concat([get_file(f) for f in all_files])
df.sort_values('post_dt', inplace=True)
print(df.size)
df.head()

175764


,post_dt,infectee_name,infectee_post_id,infector_name,infector_post_id,comment_or_submission
0,2020-03-20 19:59:59.000000,u/woodendoors7,NaN,NaN,NaN,NaN
0,2020-03-20 20:35:13.853753,u/austind9999,fl2h4gq,u/woodendoors7,fl2gay3,C
1,2020-03-20 20:37:34.242409,u/unclerico589,fl2hcd5,u/woodendoors7,fl2f39v,C
2,2020-03-20 20:40:43.198092,u/saikmat,fl2hmgw,u/woodendoors7,fl2hgh0,C
3,2020-03-20 20:45:22.757004,u/Nash_underscore,fl2hzd9,u/woodendoors7,fl2hyeu,C


In [4]:
# [row for index, row in df.iterrows() if row.infectee_name == PATIENT_0]

[post_dt                  2020-03-20 19:59:59
 infectee_name                 u/woodendoors7
 infectee_post_id                         NaN
 infector_name                            NaN
 infector_post_id                         NaN
 comment_or_submission                    NaN
 Name: 0, dtype: object]

In [50]:
# list(df.iterrows())[0][1].to_dict()
# df[:2]

,post_dt,infectee_name,infectee_post_id,infector_name,infector_post_id,comment_or_submission
0,2020-03-20 20:35:13.853753,u/austind9999,fl2h4gq,u/woodendoors7,fl2gay3,C
1,2020-03-20 20:37:34.242409,u/unclerico589,fl2hcd5,u/woodendoors7,fl2f39v,C


In [40]:
# infection_graph = nx.DiGraph()

# for index, row in df.iterrows():
#     infection_graph.add_node(row.infector_name)
#     infection_graph.add_node(row.infectee_name)
#     infection_graph.add_edge(row.infector_name, row.infectee_name, **row.to_dict())
# infection_graph.edges(data=True)

In [66]:
infectee_infector = {}
for index, row in df.iterrows():
    infectee_infector[row.infectee_name] = row.infector_name

In [5]:
len(infectee_infector)

8553

In [83]:
data = {
    'infectee': [],
    'infector': [],
    'value': []
}
processed = set()

def get_patient_0(user):
    if user not in infectee_infector or user == PATIENT_0:
        return user
    else:
        return get_patient_0(infectee_infector[user])

valid_infectors = {}
valid_infectors[np.NaN] = []
invalid_infectors = set()
valid = set()
count = 0
for index, row in df.iterrows():
    count+=1
    processed = set()
    source_pat_0 = get_patient_0(row.infector_name)
    if source_pat_0 == PATIENT_0 or (row.infector_name is np.NaN):
        if row.infector_name in valid_infectors: # and len(valid_infectors[row.infector_name]) <= 20:
            valid_infectors[row.infector_name].append(row.infectee_name)
            valid_infectors[row.infectee_name] = []
            data['infector'].append(row.infector_name if row.infector_name is not np.NaN else '')
            data['infectee'].append(row.infectee_name)
            data['value'].append(1)
    else:
        invalid_infectors.add(source_pat_0)
#     if count == 3000:
#         print(row.to_dict())
#         break

print(len(invalid_infectors))
# from pprint import pprint; pprint(data)

# fig = px.sunburst(
#     data,
#     names='infectee',
#     parents='infector',
#     values='value',
# )
# fig.show()

47


In [81]:
invalid_infectors

{'u/5amukai',
 'u/A_shovel_',
 'u/Anoobis100percent',
 'u/BeardPhile',
 'u/Chong_Long_Dong',
 'u/Drodr10',
 'u/Faustonreddit',
 'u/FivesG',
 'u/GarageInc',
 'u/GiantRobotDeadpool12',
 'u/GloryToDucks39',
 'u/ImJustDubzzz',
 'u/ItsSynnoX',
 'u/Kirby-For-Life05',
 'u/LeonidtheWill',
 'u/MapleSyrupSquirrel',
 'u/Mr_Roberge',
 'u/NotFunnyBuddy',
 'u/Pathodox',
 'u/PunntheGreat',
 'u/Sex_Shop_Souvenir',
 'u/SirQwacksAlot',
 'u/TheSCP_3008',
 'u/UnKnOwNLIKER',
 'u/Uncle_Freddy',
 'u/Yusuri-San',
 'u/_TheRope',
 'u/biggested304',
 'u/colbywankenobi0',
 'u/cyberdrugs',
 'u/drewmanchu74',
 'u/iam_inevtable',
 'u/infamouszoggy',
 'u/inslatafresca_',
 'u/jospehjoestar',
 'u/jrodjh',
 'u/kink_creamson',
 'u/longlivestheking',
 'u/mb032475',
 'u/my-name-is-blank',
 'u/neopolitam',
 'u/not-a-Shrek',
 'u/ruilybro',
 'u/shreygay123',
 'u/spiritually_deceased',
 'u/themangoperson',
 'u/yee3009'}

In [53]:
import plotly.io as pio
# pio.orca.shutdown_server()
# pio.orca.config.executable = '/home/pmr3mh/miniconda3/lib/orca_app/orca'
# pio.orca.config.server_url = None
# pio.orca.config.port = None
# pio.orca.config.save()
# pio.orca.ensure_server()
# pio.orca.config
pio.orca.status

orca status
-----------
    state: running
    executable: /home/pmr3mh/miniconda3/lib/orca_app/orca
    version: 1.2.1
    port: 44429
    pid: 2312
    command: ['/home/pmr3mh/miniconda3/lib/orca_app/orca', 'serve', '-p', '44429', '--plotly', '/home/pmr3mh/.virtualenvs/nd230-ml-tensorflow-p1/lib/python3.5/site-packages/plotly/package_data/plotly.min.js', '--graph-only', '--mathjax', 'https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js']


In [62]:
# import subprocess
# subprocess.Popen(["/home/pmr3mh/miniconda3/lib/orca_app/orca", "-p", "--plotly", "--graph-only", "--mathjax", "serve", "37587", "/home/pmr3mh/.virtualenvs/nd230-ml-tensorflow-p1/lib/python3.5/site-packages/plotly/package_data/plotly.min.js", "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js", "--keep-alive"], stdout=subprocess.PIPE, shell=True)
timestamp = datetime.utcnow().isoformat()
print(datetime.utcnow().isoformat())
fig.write_image('plot_{}.png'.format(timestamp), width=800, height=800, scale=5)

2020-03-23T00:53:48.692501


In [184]:
print(datetime.utcnow().isoformat())

2020-03-22T23:21:27.342423


In [82]:
test_infector = 'u/5amukai'
processed = set()
hier = []
def get_patient_0(user):
    hier.append(user)
    if user in processed:
        print(user)
        return None
    else:
        processed.add(user)
    if user not in infectee_infector or user == PATIENT_0:
        return user
    else:
        return get_patient_0(infectee_infector[user])
print(get_patient_0(test_infector))
print(hier)
# print(len(valid_infectors[hier[1]]))
# print(test_infector in valid_infectors[hier[1]])
print(test_infector in valid_infectors)
print(test_infector in data['infector'])
print(test_infector in data['infectee'])
df.loc[(df['infector_name'] == test_infector) | (df['infectee_name'] == test_infector)]

u/5amukai
['u/5amukai']
False
False
False


,post_dt,infectee_name,infectee_post_id,infector_name,infector_post_id,comment_or_submission
117,2020-03-25 13:30:43.952444,u/Potatoboiftw,flh6y8r,u/5amukai,flgl4zu,C
